# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('../starter_files/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 3094545808288,
 'week52high': 188.36,
 'week52low': 120.9,
 'week52highSplitAdjustOnly': 189.26,
 'week52lowSplitAdjustOnly': 120.41,
 'week52change': 0.3945360630292246,
 'sharesOutstanding': 16412922263,
 'float': 0,
 'avg10Volume': 79731462,
 'avg30Volume': 109985157,
 'day200MovingAvg': 154.14,
 'day50MovingAvg': 175.55,
 'employees': 152614,
 'ttmEPS': 11.52,
 'ttmDividendRate': 0.8995528647816116,
 'dividendYield': 0.004903964312700373,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-24',
 'nextEarningsDate': '2022-01-20',
 'peRatio': 15.73636991301685,
 'beta': 1.3712649936695398,
 'maxChangePercent': 69.8744961536718,
 'year5ChangePercent': 5.705522917443528,
 'year2ChangePercent': 1.4680475326054911,
 'year1ChangePercent': 0.4048522073481319,
 'ytdChangePercent': 0.012356089696284238,
 'month6ChangePercent': 0.294486894348009,
 'month3ChangePercent': 0.3076309087932027,
 'month1ChangePercent': 0.11330080308613857,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.4048522073481319

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,153.400,0.284178,N/A
1,AAL,19.640,0.261359,N/A
2,AAP,247.518,0.535925,N/A
3,AAPL,179.440,0.404724,N/A
4,ABBV,142.331,0.354775,N/A
...,...,...,...,...
500,YUM,140.450,0.322843,N/A
501,ZBH,134.755,-0.151028,N/A
502,ZBRA,588.430,0.573761,N/A
503,ZION,69.000,0.595025,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.780,2.316613,N/A
1,DVN,49.500,2.16537,N/A
2,F,25.571,1.882889,N/A
3,MRO,18.663,1.665822,N/A
4,MCHP,88.820,1.605293,N/A
5,FANG,122.030,1.506145,N/A
6,NUE,127.420,1.276483,N/A
7,NVDA,296.460,1.273934,N/A
8,FTNT,328.070,1.271241,N/A
9,EXR,221.340,1.115312,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000
1000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\Local User\anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.780,2.316613,0
1,DVN,49.500,2.16537,0
2,F,25.571,1.882889,0
3,MRO,18.663,1.665822,1
4,MCHP,88.820,1.605293,0
5,FANG,122.030,1.506145,0
6,NUE,127.420,1.276483,0
7,NVDA,296.460,1.273934,0
8,FTNT,328.070,1.271241,0
9,EXR,221.340,1.115312,0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
hqm_dataframe.dropna(inplace=True)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.47505
1      0.459082
2      0.798403
3      0.654691
4      0.588822
         ...   
500    0.542914
501    0.031936
502    0.838323
503    0.854291
504    0.622754
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.305389
1      0.091816
2      0.646707
3      0.918164
4      0.804391
         ...   
500    0.756487
501    0.035928
502    0.516966
503    0.894212
504    0.720559
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.165669
1      0.035928
2      0.706587
3      0.956088
4      0.922156
         ...   
500    0.608782
501    0.033932
502    0.760479
503    0.313373
504    0.738523
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.177645
1      0.908184
2      0.343313
3      0.782435
4      0.884232
         ...   
500    0.694611
501    0.530938
502    0.091816
503     0.45509
504    0.147705
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.040,N/A,0.290342,0.47505,0.018763,0.305389,-0.006596,0.165669,0.014305,0.177645,N/A
1,AAL,19.990,N/A,0.269522,0.459082,-0.115025,0.091816,-0.111516,0.035928,0.149137,0.908184,N/A
2,AAP,245.891,N/A,0.537521,0.798403,0.137163,0.646707,0.154958,0.706587,0.050609,0.343313,N/A
3,AAPL,179.940,N/A,0.405963,0.654691,0.290167,0.918164,0.300858,0.956088,0.114521,0.782435,N/A
4,ABBV,137.500,N/A,0.357904,0.588822,0.206148,0.804391,0.260473,0.922156,0.138431,0.884232,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.680,N/A,0.330793,0.542914,0.188053,0.756487,0.123971,0.608782,0.099938,0.694611,N/A
501,ZBH,135.593,N/A,-0.145578,0.031936,-0.203625,0.035928,-0.115499,0.033932,0.077884,0.530938,N/A
502,ZBRA,583.880,N/A,0.575552,0.838323,0.092465,0.516966,0.175026,0.760479,-0.001195,0.091816,N/A
503,ZION,69.130,N/A,0.602126,0.854291,0.258851,0.894212,0.042021,0.313373,0.067226,0.45509,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,98.19,199,0.444090,0.885149,0.147456,0.837624,0.221461,0.473267,0.093820,0.530693,0.681683
1,AAL,13.89,1411,-0.526494,0.0237624,-0.564540,0.0158416,0.509431,0.908911,0.172430,0.827723,0.444059
2,AAP,161.13,121,0.088066,0.578218,0.148378,0.839604,0.295700,0.643564,0.144608,0.742574,0.70099
3,AAPL,467.65,41,1.171724,0.992079,0.401695,0.968317,0.474900,0.887129,0.189840,0.879208,0.931683
4,ABBV,97.29,201,0.478770,0.89703,0.001711,0.629703,0.077880,0.19802,-0.024533,0.0693069,0.448515
5,ABC,104.97,186,0.163705,0.651485,0.100112,0.774257,0.241867,0.522772,0.066990,0.364356,0.578218
6,ABMD,305.78,64,0.546138,0.942574,0.874783,0.99802,0.645795,0.954455,0.145153,0.746535,0.910396
7,ABT,102.94,190,0.161073,0.647525,0.139130,0.823762,0.093917,0.233663,0.081815,0.457426,0.540594
8,ACN,237.88,82,0.197484,0.69505,0.085519,0.762376,0.279570,0.605941,0.067045,0.366337,0.607426
9,ADBE,453.91,43,0.532839,0.934653,0.196705,0.879208,0.252382,0.550495,0.006829,0.150495,0.628713


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()